# Обучение модели по генерации текста с использованием Hugging Face

__Задача__: обучить модель (дообучить существующую) для генерации текста, являющегося ответом на диалоговое сообщение 

In [1]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import datasets
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration
from transformers import T5TokenizerFast
from transformers import pipeline
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Импорт датасета

In [4]:
dataset = datasets.load_dataset('Den4ikAI/russian_dialogues')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/951 [00:00<?, ?B/s]

dataset.jsonl:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2477321 [00:00<?, ? examples/s]

In [20]:
# leave only 0.5% of dataset

k = 0.005
cropped_dataset = dataset['train'].select(
    [i for i in range(int(len(dataset['train']) * k))]
    )
cropped_dataset

Dataset({
    features: ['question', 'answer', 'relevance'],
    num_rows: 12386
})

In [21]:
# example
for i in range(10):
    print(f"{i+1}) {cropped_dataset[i]['question']} ----> {cropped_dataset[i]['answer']}")

1) как дела? ----> там хорошо
2) вы кефир пачему не кушаете, не любите? ----> я ряженку лучше люблю.
3) если в расходную накладную забить дури и выкурить, то получится приходный документ? ----> особенно когда придет комиссия проверять документацию
4) покажись в шапке ----> ды щаз приветик
5) давай не будем об этом ----> давай поговорим о чем-нибудь другом
6) препарат для лечения сильно понижает давление. что порекомендуете? ----> чтоб не сильно? или что? препарат принимай и кофе пей
7) мужчина, если ты занюхиваешь волосами соседки, то какой аромат предпочитаешь? ----> предпочитаю соседкиными пирогами закусывать. -
8) можете ли вы с ходу отличить японских фигуристов от китайских без представления? ----> японцы в целом куда симпатичнее китайцев. но конечно же бывают исключения во всем
9) как стать наемником? ----> свяжись с нанимателями, заключи договор и будь наемником.
10) слышала, что есть крем от шрамов. кто знает, скажите пожалуйста название. и поможет ли он от шрамов которым лет. д

Видно, что представленные в датасете диалоги весьма специфические

In [22]:
# split to train, validate, test
train_size = 0.8
validate_size = 0.15
test_size = 0.05

cropped_dataset = datasets.DatasetDict({
    'train': cropped_dataset.select([i for i in range(int(len(cropped_dataset) * train_size))]),
    'validation': cropped_dataset.select([i for i in range(int(len(cropped_dataset) * validate_size))]),
    'test': cropped_dataset.select([i for i in range(int(len(cropped_dataset) * test_size))])
    })

cropped_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'relevance'],
        num_rows: 9908
    })
    validation: Dataset({
        features: ['question', 'answer', 'relevance'],
        num_rows: 1857
    })
    test: Dataset({
        features: ['question', 'answer', 'relevance'],
        num_rows: 619
    })
})

Сделаю тестовую выборку небольшой, потому что вычисление ошибки на тестовой выборке будет долгим в силу того, что генерация текста сама по себе не происходит моментально

## Импорт модели и токенайзера. Токенизация датасета

In [11]:
# предобученная модель T5 для различных задач на русском языке
model_name = "cointegrated/rut5-base-multitask"

model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5TokenizerFast.from_pretrained(model_name)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting

In [23]:
def remove_None_values(dataset):
    d = dict()
    for key in dataset.keys():
        arr = []
        for elem in dataset[key]:
            if (elem['question'] is None) or (elem['answer'] is None):
                arr.append({'question': '',
                            'answer': '',
                            'relevance': elem['relevance']})
            else:
                arr.append({'question': elem['question'],
                            'answer': elem['answer'],
                            'relevance': elem['relevance']})
        d[key] = datasets.Dataset.from_list(arr)

    return datasets.DatasetDict(d)

# function to get tokenized input for model
def tokenize_function(example):
    max_length = 128 # I don't think we need more to make answers in dialog

    tokenized_input = tokenizer(['continue dialog | ' + sub_example for sub_example in example['question']], add_special_tokens=False,
                                padding=True, truncation=True, max_length=max_length,
                                return_tensors='pt')
    tokenized_output = tokenizer(example['answer'], add_special_tokens=False,
                                 padding=True, truncation=True, max_length=max_length,
                                 return_tensors='pt')

    tokenized_input['labels'] = tokenized_output['input_ids']
    tokenized_input['decoder_attention_mask'] = tokenized_output['attention_mask']

    return tokenized_input

In [24]:
cropped_dataset = remove_None_values(cropped_dataset)

In [25]:
tokenized_dataset = cropped_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9908 [00:00<?, ? examples/s]

Map:   0%|          | 0/1857 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

## Обучение

Посмотрим на работу модели. Один из предобученных тегов - answer, с ним модель генерирует ответ (модель предобучена на ответах mail.ru)

In [26]:
def generate(instruction, dialog):
    input_ids = tokenizer(f"{instruction} | {dialog}",
                          return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return output

In [27]:
text1 = 'Привет, я Алекс. Мне нравится заниматься спортом и играть в игры. А тебе?'
text2 = 'Привет, я Алекс. Мне нравится заниматься спортом и играть в игры.'
text3 = 'Привет, я Алекс.'

print(generate('answer', text1))
print(generate('answer', text2))
print(generate('answer', text3))

Мне нравится играть в игры.
Это я и хочу.
я знаю, что я тебе сказал.


Перед началом обучения вычислим ошибки модели на тестовых текстах с использованием тега "answer" и сохраним вместе с предсказаниями модели. Далее после обучения мы сравним их с новыми результатами.

In [28]:
def calculate_preds_and_losses(input_texts, output_texts, instruction):
    preds = []
    losses = []

    i = 0
    for input_text, output_text in zip(input_texts, output_texts):
        max_length = 128 # I don't think we need more to make answers in dialog

        pred_text = generate(instruction, input_text)

        tokenized_input = tokenizer(output_text, add_special_tokens=False,
                                    padding=True, truncation=True, max_length=max_length)
        tokenized_output = tokenizer(pred_text, add_special_tokens=False,
                                    padding=True, truncation=True, max_length=max_length)

        if tokenized_input['input_ids'] == []:
            tokenized_input['input_ids'] = [0]
            tokenized_input['attention_mask'] = [1]
        if tokenized_output['input_ids'] == []:
            tokenized_output['input_ids'] = [0]
            tokenized_output['attention_mask'] = [1]

        tokenized_input = {
            'input_ids': torch.tensor([tokenized_input['input_ids']]),
            'attention_mask': torch.tensor([tokenized_input['attention_mask']])
        }
        tokenized_output = {
            'input_ids': torch.tensor([tokenized_output['input_ids']]),
            'attention_mask': torch.tensor([tokenized_output['attention_mask']])
        }

        loss = model(
            input_ids = tokenized_input['input_ids'].to(device),
            attention_mask = tokenized_input['attention_mask'].to(device),
            labels = tokenized_output['input_ids'].to(device),
            decoder_attention_mask = tokenized_output['attention_mask'].to(device),
            return_dict = True
        ).loss

        preds.append(pred_text)
        losses.append(loss.item())

        i += 1


        if (i+1) % 100 == 0:
            print(f"{i+1} / {len(input_texts)}")

    return preds, losses

In [29]:
input_texts = cropped_dataset['test']['question']
output_texts = cropped_dataset['test']['answer']

preds_before, losses_before = calculate_preds_and_losses(input_texts, output_texts, 'answer')

100 / 619
200 / 619
300 / 619
400 / 619
500 / 619
600 / 619


Подготовим параметры и начнем обучение. На вход модели будут подаваться текста с добавленной строкой "continue dialog | " в начале. (это изменение было внесено в процессе токенизации датасета). Таким образом, мы сможем сравнить результаты с предыдущим тегом ("answer | ")

In [30]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [31]:
training_args = TrainingArguments(
    output_dir="test_TextGeneration",
    eval_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.05,
    save_strategy='epoch'
)

In [32]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.656000,0.533154
2,0.600900,0.528030


TrainOutput(global_step=1240, training_loss=0.6236657419512349, metrics={'train_runtime': 1423.4083, 'train_samples_per_second': 13.922, 'train_steps_per_second': 0.871, 'total_flos': 3367415961354240.0, 'train_loss': 0.6236657419512349, 'epoch': 2.0})

Обучение длилось 23 минуты на Google Colab T4

## Оценка результатов

Получим новые предсказания и значения ошибок

In [34]:
input_texts = cropped_dataset['test']['question']
output_texts = cropped_dataset['test']['answer']

preds_after, losses_after = calculate_preds_and_losses(input_texts, output_texts, 'continue dialog')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


100 / 619
200 / 619
300 / 619
400 / 619
500 / 619
600 / 619


Сравним парочку предсказаний, а также средние показатели ошибки

In [35]:
print("Mean losses")
print(f"Before: {np.mean(losses_before)}")
print(f"After: {np.mean(losses_after)}")

Mean losses
Before: 3.135417084123861
After: 1.978448785564626


In [36]:
start = 0
end = 5

real_texts = input_texts
for pred_before, pred_after, loss_before, loss_after, real_text in zip(preds_before[start : end],
                                                                       preds_after[start : end],
                                                                       losses_before[start : end],
                                                                       losses_after[start : end],
                                                                       real_texts[start : end]):
    print(f"Real text: {real_text}")
    print(f"Before: {pred_before} | loss: {loss_before}")
    print(f"After: {pred_after} | loss: {loss_after}")
    print("==========================================")

Real text: как дела?
Before: а я знаю, что я знаю, что я знаю. | loss: 4.0995659828186035
After: я знаю, что я хочу жить. а я  | loss: 2.249361991882324
Real text: вы кефир пачему не кушаете, не любите?
Before: а я тебе её не хочу пить. | loss: 2.7725930213928223
After: я только кефир и кефир хочу. а  | loss: 2.35430908203125
Real text: если в расходную накладную забить дури и выкурить, то получится приходный документ?
Before: если я хочу купить и купить - я хоч | loss: 3.1573386192321777
After: если я хочу, то я хочу. а если | loss: 1.8391541242599487
Real text: покажись в шапке
Before: а я тебе сказал, что я тебе сказал, что  | loss: 3.0216856002807617
After: я хочу и я хочу и я хочу | loss: 1.4499293565750122
Real text: давай не будем об этом
Before: а я тебе сказал, что я тебе сказал, что  | loss: 2.1996512413024902
After: давай не будем об этом говорить. а я уже говорил | loss: 1.6043914556503296


Согласно ошибке, результат действительно улучшился. Думаю, можно было бы добиться больших результатов при увеличении числа эпох, числа пакета (T4 не может обрабатывать больше, чем 16 текстов за раз) и размера датасета (я взял от него лишь 0.5%).